<a href="https://colab.research.google.com/github/luc-leonard/taming-transformers/blob/master/scripts/clip_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK PREPARATION


In [1]:
!git clone https://github.com/luc-leonard/taming-transformers
!git clone https://github.com/openai/CLIP

Cloning into 'taming-transformers'...
remote: Enumerating objects: 750, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 750 (delta 43), reused 64 (delta 27), pack-reused 658
Receiving objects: 100% (750/750), 118.83 MiB | 42.77 MiB/s, done.
Resolving deltas: 100% (178/178), done.
Cloning into 'CLIP'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


In [2]:
!pip install kornia omegaconf pytorch-lightning ftfy

     |████████████████████████████████| 286kB 29.6MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 808kB 55.3MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 645kB 39.6MB/s 
     |████████████████████████████████| 112kB 59.2MB/s 
     |████████████████████████████████| 829kB 48.0MB/s 
     |████████████████████████████████| 276kB 47.6MB/s 
     |████████████████████████████████| 112kB 60.0MB/s 
     |████████████████████████████████| 10.6MB 32.8MB/s 
     |████████████████████████████████| 1.3MB 45.5MB/s 
     |████████████████████████████████| 296kB 47.6MB/s 
     |████████████████████████████████| 143kB 58.1MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=195ccfe6e007d5d20e66c096bd31c62eaeda94cb667aa0456ddb1c95e0aed9fe
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
  Created wheel for

## DOWNLOADING VQGAN MODEL

In [3]:
!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' > vqgan_imagenet_f16_16384.yaml
!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' > vqgan_imagenet_f16_16384.ckpt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   692  100   692    0     0   1338      0 --:--:-- --:--:-- --:--:--  1338
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  934M  100  934M    0     0  14.9M      0  0:01:02  0:01:02 --:--:-- 14.9M


## imports

In [5]:
import sys
sys.path.append('taming-transformers')
sys.path.append('CLIP')

# HAVE WE BEEN LUCKY ? :)

In [4]:
!nvidia-smi

Fri Jun 11 11:09:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# RUNNING

/!\ DO NOT FORGET TO GO TO 'runtime' => 'change runtime type' and to choose a GPU instance !

In [6]:
#@title Example form fields
#@markdown Forms support many types of fields.

prompt = 'A angry Mario'  #@param {type: "string"}
epochs = 240  #@param {type: "slider", min: 100, max: 1000}
#@markdown ---


In [7]:
from scripts.clip_generator.trainer import Trainer
from scripts.clip_generator.dreamer import load_vqgan_model
import clip

device = 'cuda'


vqgan_model = load_vqgan_model('./vqgan_imagenet_f16_16384.yaml', './vqgan_imagenet_f16_16384.ckpt')
clip_model = clip.load('ViT-B/32', jit=False)[0].eval().requires_grad_(False).to(device)

trainer = Trainer([prompt], vqgan_model, clip_model, device=device)
trainer.start()

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


8.19kB [00:00, 510kB/s]                    


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


  1%|▎                                     | 3.16M/354M [00:00<00:11, 31.6MiB/s]

...

i: 250, loss: 0.671451, losses: 0.671451


/ |                                      #          | 300 Elapsed Time: 0:04:20

i: 300, loss: 0.667589, losses: 0.667589


- |                            #                    | 301 Elapsed Time: 0:04:21

KeyboardInterrupt: ignored